# DUDI-WEEK4 : SQL with Python - SQLite
---

## 학습목표
- python을 이용하여 데이터베이스에 원하는 데이터를 읽어오고 이를 csv 파일로 저장할 수 있다.
- pandas를 이용한 데이터 처리와 sql을 이용한 데이터 처리의 효율성을 비교한다.

In [ ]:
!pip install pandas fiona shapely pyproj rtree geopandas

In [ ]:
import sqlite3
import pandas as pd
import time
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

db_path = "./drive/MyDrive/aiffel/강남/풀잎/두디/NYU Taxi.db"

## 1. Python으로 SQLite DB 읽어오기
---

- select query 결과를 DataFrame로 만드는 방법

In [ ]:
con = sqlite3.connect(db_path)
cursor = con.cursor()

In [ ]:
# 실행을 원하는 쿼리를 작성하여 보낸다.
query = cursor.execute("SELECT * FROM taxi_trips LIMIT 100;")

# 위에서 실행한 쿼리 결과의 features
query.description

(('vendor_id', None, None, None, None, None, None),
 ('pickup_datetime', None, None, None, None, None, None),
 ('dropoff_datetime', None, None, None, None, None, None),
 ('passenger_count', None, None, None, None, None, None),
 ('pickup_longitude', None, None, None, None, None, None),
 ('pickup_latitude', None, None, None, None, None, None),
 ('dropoff_longitude', None, None, None, None, None, None),
 ('dropoff_latitude', None, None, None, None, None, None),
 ('pickup_location_id', None, None, None, None, None, None),
 ('dropoff_location_id', None, None, None, None, None, None))

In [ ]:
cols = [column[0] for column in query.description]

# 쿼리 결과의 features를 column name으로 삼아 DataFrame를 생성한다.
res_df = pd.DataFrame.from_records(data=query.fetchall(), 
                                   columns=cols)
res_df

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_location_id,dropoff_location_id
0,1,2016-01-04 12:24:17 UTC,2016-01-04 13:01:48 UTC,0,-73.815224,40.700081,-73.950699,40.755222,130.0,193.0
1,1,2016-05-23 23:37:37 UTC,2016-05-23 23:37:45 UTC,0,-73.861633,40.705029,-73.861633,40.705029,102.0,102.0
2,1,2016-02-07 17:23:48 UTC,2016-02-07 17:23:57 UTC,0,-73.954941,40.687866,-73.954742,40.687862,17.0,17.0
3,1,2016-02-18 21:26:15 UTC,2016-02-18 21:28:00 UTC,0,-73.983543,40.716404,-73.983543,40.716404,232.0,232.0
4,1,2016-02-28 01:08:48 UTC,2016-02-28 01:09:01 UTC,0,-74.035942,40.736488,-74.035942,40.736492,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,1,2016-06-21 17:04:05 UTC,2016-06-21 17:10:48 UTC,1,-73.834335,40.766262,-73.834229,40.766094,92.0,92.0
96,1,2016-04-27 12:17:28 UTC,2016-04-27 12:31:18 UTC,1,-73.989983,40.714088,-73.963203,40.757687,232.0,229.0
97,1,2016-06-11 01:51:01 UTC,2016-06-11 01:57:45 UTC,1,-73.996368,40.690723,-73.986618,40.703182,52.0,66.0
98,1,2016-05-16 12:05:12 UTC,2016-05-16 12:32:00 UTC,1,-74.013687,40.712769,-73.983894,40.752453,261.0,164.0


## 2. 실습
---

비어져 있는 query를 작성하여 문제를 해결하여 봅시다. 실습 결과물로 완성된 DataFrame는 `이름_문제번호.csv` 형식으로 저장하여 노션에 올려주세요.

DataFrame를 csv로 저장하는 방법에 대해서는 [이곳](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)을 참고하여 주세요.

### 1) 실습 데이터

실습에 사용되는 데이터베이스는 캐글 사이트에 올라와 있는 New York City Taxi Trip Duration(https://www.kaggle.com/c/nyc-taxi-trip-duration) 데이터 셋과 GCP public dataset(bigquery-public-data.new_york_taxi_trips.taxi_zone_geom)을 결합하여 생성한 데이터 셋입니다.

데이터베이스에는 2가지의 테이블이 아래와 같이 정의되어 있습니다.

- taxi_trips : 택시 운행 정보
```sql
CREATE TABLE "taxi_trips" (
	"vendor_id"	INTEGER,
	"pickup_datetime"	TEXT,
	"dropoff_datetime"	TEXT,
	"passenger_count"	INTEGER,
	"pickup_longitude"	REAL,
	"pickup_latitude"	REAL,
	"dropoff_longitude"	REAL,
	"dropoff_latitude"	NUMERIC,
	"pickup_location_id"	INTEGER,
	"dropoff_location_id"	INTEGER
);
```

- taxi_zone_geom : taxi_trips에 나타나는 구역(pickup_location_id, dropoff_location_id) 정보
```sql
CREATE TABLE "taxi_zone_geom" (
	"zone_id"	INTEGER,
	"zone_name"	TEXT,
	"borough"	TEXT,
	"zone_geom"	TEXT
);
```

### 2) 사람들이 가장 택시를 많이 타는 곳 top 100은?
- 출력 column : zone_id, zone_name, borough, cnt, zone_geom
    - zone_id : 구역의 id
    - zone_name : 구역이 이름
    - borough : 구역의 속하는 지역
    - cnt : 해당 구역에서 출발한 trip의 수
    - zone_geom : 구역의 geometry 정보

✨ 출력 예시 - 데이터를 모두 읽어와 pandas에서 처리하는 경우

In [ ]:
def logging_time(original_fn):
    def wrapper_fn(*args, **kwargs):
        start_time = time.time()
        result = original_fn(*args, **kwargs)
        end_time = time.time()
        print("WorkingTime[{}]: {} sec".format(original_fn.__name__, end_time-start_time))
        return result
    return wrapper_fn

In [ ]:
def execute_query(q):
    """
    query를 실행시키고 그 결과를 pnadas.DataFrame에 담아 반환합나다.
    """
    con = sqlite3.connect(db_path)
    cursor = con.cursor()

    query = cursor.execute(q)
    cols = [column[0] for column in query.description]
    res = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)
    # display(res)

    con.close()
    return res

In [ ]:
@logging_time
def get_top100_zone(target):
    """
    db에서 테이블의 모든 정보를 가져 온 후, 
    target의 등장 빈도가 가장 높은 100개의 tuple를 반환합니다.
    이때, 반환되는 tuple의 정보는 zone_id, zone_name, borough, cnt, zone_geom 입니다.
    """
    # 1. 두개 table 정보를 모두 읽어 dataframe로 할당한다
    taxi_trips_df = execute_query("SELECT * FROM taxi_trips;")
    # display(taxi_trips_df.info())

    taxi_zone_geom_df = execute_query("SELECT * FROM taxi_zone_geom;")
    # display(taxi_zone_geom_df.info())
    
    # 2. taxi_trips_df를 target을 기준으로 
    #    grouping하여 target별 column 수를 계산한다.
    #    이때, pickup_datetime을 사용한 이유는 결측값이 없는 임의의 column을 선정하였기 때문이다.
    top100 = taxi_trips_df[['pickup_datetime', target]].groupby(by=[target]) # target를 기준으로 그룹화
    top100 = top100.count()  # aggrigation func로 count 사용
    top100 = top100.sort_values(by='pickup_datetime', ascending=False) # count 정보를 기준으로 내림차순
    top100 = top100.head(100) # 상위 10개만을 추출
    top100 = top100.reset_index() # index로 되어 있는 target를 column으로 변환
    top100 = top100.rename(columns={'index':target, 'pickup_datetime':'cnt'}) # column 이름을 변경, index → target, pickup_datetime → cnt
    # display(top100)
    
    # 3. taxi_zone_geom_df와 top10을 merge
    #    merge 기준은 taxi_zone_geom_df.zone_id와 top100.target이다.
    #    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html
    res = top100.merge(taxi_zone_geom_df,
                      how='inner',
                      left_on=target,
                      right_on='zone_id')
    res = res[['zone_id', 'zone_name', 'borough', 'cnt', 'zone_geom']]
    # display(res)
    
    
    return res

In [ ]:
res = get_top100_zone('pickup_location_id')
res

WorkingTime[get_top100_zone]: 12.125190019607544 sec


,zone_id,zone_name,borough,cnt,zone_geom
0,237,Upper East Side South,Manhattan,55732,"POLYGON((-73.9656696 40.7628045559999, -73.965..."
1,161,Midtown Center,Manhattan,53293,"POLYGON((-73.9748872719999 40.7559237909999, -..."
2,236,Upper East Side North,Manhattan,51259,"POLYGON((-73.9572940999999 40.7742835549999, -..."
3,186,Penn Station/Madison Sq West,Manhattan,49577,"POLYGON((-73.9905176129999 40.7460386379999, -..."
4,162,Midtown East,Manhattan,49281,"POLYGON((-73.970759194 40.7558248509999, -73.9..."
...,...,...,...,...,...
95,247,West Concourse,Bronx,258,"POLYGON((-73.9119704559999 40.8431043939999, -..."
96,10,Baisley Park,Queens,246,"POLYGON((-73.7840990399998 40.6896477549999, -..."
97,83,Elmhurst/Maspeth,Queens,245,"POLYGON((-73.8913914549998 40.7466410039998, -..."
98,70,East Elmhurst,Queens,217,"POLYGON((-73.8591354529999 40.7614184409999, -..."


In [ ]:
def show_map(df, trg, tooltip_trg):
    """
    입력되는 pandas.DataFrame를 folium의 지도 위도 출력합니다.
    """
    gs = gpd.GeoSeries.from_wkt(df['zone_geom'])
    gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")

    f = folium.Figure(width=600, height=500)
    m = folium.Map(location=[40.75, -73.90], zoom_start=10, tiles='CartoDB positron').add_to(f)
    choropleth = folium.Choropleth(geo_data=gdf, 
                    neme='zone_name',
                    data=gdf,
                    columns=['zone_name', trg],
                    key_on='feature.properties.zone_name',
                    fill_color='YlGn',
                    fill_opacity=0.7,
                    line_opacity=0.2,
                    legend_name=trg).add_to(m)

    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}

    tooltip = folium.GeoJson(
        gdf,
        style_function=style_function, 
        control=False,
        tooltip=folium.GeoJsonTooltip(
            fields=tooltip_trg,
            localize=True
        )
    )

    choropleth.add_child(tooltip)
    display(f)

In [ ]:
show_map(res, 'cnt', ['borough', 'zone_name'])

Output hidden; open in https://colab.research.google.com to view.

✨ 실습 - 쿼리를 활용하여 문제를 해결해보자!

In [ ]:
@logging_time
def solution_with_query(q):    
    return execute_query(q)

In [ ]:
query = \
"""
select top10.pickup_location_id as zone_id,
       zone.zone_name, zone.borough, top10.cnt, zone.zone_geom
from (select pickup_location_id, count(pickup_location_id) as cnt
      from taxi_trips
      group by pickup_location_id
      order by count(pickup_location_id) desc 
      LIMIT 100) as top10
left join taxi_zone_geom as zone
on top10.pickup_location_id=zone.zone_id
"""
res = solution_with_query(query)
res

WorkingTime[solution_with_query]: 4.088299036026001 sec


,zone_id,zone_name,borough,cnt,zone_geom
0,237,Upper East Side South,Manhattan,55732,"POLYGON((-73.9656696 40.7628045559999, -73.965..."
1,161,Midtown Center,Manhattan,53293,"POLYGON((-73.9748872719999 40.7559237909999, -..."
2,236,Upper East Side North,Manhattan,51259,"POLYGON((-73.9572940999999 40.7742835549999, -..."
3,186,Penn Station/Madison Sq West,Manhattan,49577,"POLYGON((-73.9905176129999 40.7460386379999, -..."
4,162,Midtown East,Manhattan,49281,"POLYGON((-73.970759194 40.7558248509999, -73.9..."
...,...,...,...,...,...
95,247,West Concourse,Bronx,258,"POLYGON((-73.9119704559999 40.8431043939999, -..."
96,10,Baisley Park,Queens,246,"POLYGON((-73.7840990399998 40.6896477549999, -..."
97,83,Elmhurst/Maspeth,Queens,245,"POLYGON((-73.8913914549998 40.7466410039998, -..."
98,70,East Elmhurst,Queens,217,"POLYGON((-73.8591354529999 40.7614184409999, -..."


In [ ]:
show_map(res, 'cnt', ['borough', 'zone_name'])

Output hidden; open in https://colab.research.google.com to view.

### 2) 사람들이 가장 택시를 많이 내리는 곳 top 100은?
- 출력 column : zone_id, zone_name, borough, cnt, zone_geom
    - zone_id : 구역의 id
    - zone_name : 구역이 이름
    - borough : 구역의 속하는 지역
    - cnt : 해당 구역에서 출발한 trip의 수
    - zone_geom : 구역의 geometry 정보

✨ 출력 예시 - 데이터를 모두 읽어와 pandas에서 처리하는 경우

In [ ]:
res = get_top100_zone('dropoff_location_id')
res

WorkingTime[get_top100_zone]: 11.602027654647827 sec


,zone_id,zone_name,borough,cnt,zone_geom
0,161,Midtown Center,Manhattan,55591,"POLYGON((-73.9748872719999 40.7559237909999, -..."
1,236,Upper East Side North,Manhattan,51675,"POLYGON((-73.9572940999999 40.7742835549999, -..."
2,237,Upper East Side South,Manhattan,48454,"POLYGON((-73.9656696 40.7628045559999, -73.965..."
3,170,Murray Hill,Manhattan,46525,"POLYGON((-73.971944289 40.7459318719999, -73.9..."
4,230,Times Sq/Theatre District,Manhattan,44737,"POLYGON((-73.9831679599999 40.7561402219999, -..."
...,...,...,...,...,...
95,168,Mott Haven/Port Morris,Bronx,1163,MULTIPOLYGON(((-73.8993874857103 40.8019356650...
96,228,Sunset Park West,Brooklyn,1134,"POLYGON((-73.9969646509999 40.6692345489999, -..."
97,198,Ridgewood,Queens,1042,"POLYGON((-73.908505011 40.7174689959999, -73.9..."
98,12,Battery Park,Manhattan,1036,"POLYGON((-74.015655822 40.7048826339999, -74.0..."


In [ ]:
show_map(res, 'cnt', ['borough', 'zone_name'])

Output hidden; open in https://colab.research.google.com to view.

### 3) 등장빈도가 높은 여행경로 100개를 내림차순으로 정렬하시오.
- 출력 column : pickup_zone, dropoff_zone, count
    - pickup_zone : trip의 시작 지점
    - dropoff_zone : trip의 종료 지점
    - cnt : 해당 trip의 등장 횟수

✨ 출력 예시 - 데이터를 모두 읽어와 pandas에서 처리하는 경우

In [ ]:
@logging_time
def get_frequent_routine():
    """
    db에서 테이블의 모든 정보를 가져 온 후, 
    target의 등장 빈도가 가장 높은 100개의 tuple를 반환합니다.
    이때, 반환되는 tuple의 정보는 zone_id, zone_name, borough, cnt, zone_geom 입니다.
    """
    # 1. taxi_trips 정보를 모두 읽어 dataframe로 할당한다.
    taxi_trips_df = execute_query("SELECT * FROM taxi_trips WHERE pickup_location_id IS NOT NULL AND dropoff_location_id IS NOT NULL;")
    # display(taxi_trips_df.info())

    # 2. taxi_trips_df에서 pickup_location_id와 dropoff_location_id를 연결하여
    #    새로운 column `routine`를 정의한다.
    taxi_trips_df['routine'] = taxi_trips_df.apply(lambda x : str(x['pickup_location_id']) + " " + str(x['dropoff_location_id']), axis=1)

    
    # 3. taxi_trips_df에서 routine를 기준으로 
    #    grouping하여 routine별 column 수를 계산한다.
    #    이때, pickup_datetime을 사용한 이유는 결측값이 없는 임의의 column을 선정하였기 때문이다.
    top100 = taxi_trips_df[['pickup_datetime', 'routine']].groupby(by=['routine']) # routine을 기준으로 그룹화
    top100 = top100.count()  # aggrigation func로 count 사용
    top100 = top100.sort_values(by='pickup_datetime', ascending=False) # count 정보를 기준으로 내림차순
    top100 = top100.head(100) # 상위 100개만을 추출
    top100 = top100.reset_index() # index로 되어 있는 target를 column으로 변환
    top100 = top100.rename(columns={'index':'routine', 'pickup_datetime':'cnt'})

    # 4. 출력 형식을 맞추기 위하여 `routine` column을 분해하여
    #    `pickup_zone`과 `dropoff_zone`을 생성한다.
    top100['pickup_zone'] = top100.apply(lambda x : x['routine'].split()[0], axis=1)
    top100['dropoff_zone'] = top100.apply(lambda x : x['routine'].split()[1], axis=1)
    res = top100[['pickup_zone', 'dropoff_zone', 'cnt']]

    # display(res)
    return res

In [ ]:
res = get_frequent_routine()
res

WorkingTime[get_frequent_routine]: 31.690722227096558 sec


,pickup_zone,dropoff_zone,cnt
0,237,236,7308
1,236,237,6385
2,236,236,6314
3,237,237,5701
4,239,238,3743
...,...,...,...
95,186,48,1870
96,164,170,1854
97,186,246,1851
98,148,79,1845


✨ 실습 - 쿼리를 활용하여 문제를 해결해보자!
- tip ) http://www.devkuma.com/books/pages/1347

In [ ]:
query = \
"""
select substr(routine, 1, pos-1) as pickup_zone, 
       substr(routine, pos+1) as dropoff_zone,
       count(routine) as cnt
from (select (pickup_location_id || ' ' || dropoff_location_id) as routine,
			  length(pickup_location_id) as pos
	  from taxi_trips
      where pickup_location_id is not null and dropoff_location_id is not null)
group by routine
order by count(routine) desc
limit 100
"""

res = solution_with_query(query)
res

WorkingTime[solution_with_query]: 5.853894948959351 sec


,pickup_zone,dropoff_zone,cnt
0,23,236,7308
1,23,237,6385
2,23,236,6314
3,23,237,5701
4,23,238,3743
...,...,...,...
95,18,48,1870
96,16,170,1854
97,18,246,1851
98,14,79,1845


### 4) 출발구역에 따른 여행객 수의 비율
- 출력 column : zone_id, zone_name, borough, passenger_1, passenger_2, passenger_3, passenger_4, passenger_other, zone_geom
    - zone_id : 구역의 id
    - zone_name : 구역이 이름
    - borough : 구역의 속하는 지역
    - cnt : 해당 구역에서 출발한 trip의 수
    - passenger_1, passenger_2, passenger_3, passenger_4는 passenger_count에서 오며 각 element는 해당 zone에서 출발한 여행에서 해당 여행객수의 비율(1이하의 float)이다. passenger_other는 1\~4 이외의 여행객 수(0, 5\~9)에 대한 비율이다.

✨ 출력 예시 - 데이터를 모두 읽어와 pandas에서 처리하는 

In [ ]:
@logging_time
def get_top100_zone(target):
    """
    db에서 테이블의 모든 정보를 가져 온 후, 
    target의 등장 빈도가 가장 높은 100개의 tuple를 반환합니다.
    이때, 반환되는 tuple의 정보는 zone_id, zone_name, borough, cnt, zone_geom 입니다.
    """
    # 1. 두개 table 정보를 모두 읽어 dataframe로 할당한다
    taxi_trips_df = execute_query("SELECT * FROM taxi_trips;")
    # display(taxi_trips_df.info())

    taxi_zone_geom_df = execute_query("SELECT * FROM taxi_zone_geom;")
    # display(taxi_zone_geom_df.info())
    
    # 2. taxi_trips_df를 target을 기준으로 
    #    grouping하여 target별 column 수를 계산한다.
    #    이때, pickup_datetime을 사용한 이유는 결측값이 없는 임의의 column을 선정하였기 때문이다.
    top100 = taxi_trips_df[['pickup_datetime', target]].groupby(by=[target]) # target를 기준으로 그룹화
    top100 = top100.count()  # aggrigation func로 count 사용
    top100 = top100.sort_values(by='pickup_datetime', ascending=False) # count 정보를 기준으로 내림차순
    top100 = top100.head(100) # 상위 10개만을 추출
    top100 = top100.reset_index() # index로 되어 있는 target를 column으로 변환
    top100 = top100.rename(columns={'index':target, 'pickup_datetime':'cnt'}) # column 이름을 변경, index → target, pickup_datetime → cnt
    # display(top100)
    
    # 3. taxi_zone_geom_df와 top10을 merge
    #    merge 기준은 taxi_zone_geom_df.zone_id와 top100.target이다.
    #    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html
    res = top100.merge(taxi_zone_geom_df,
                      how='inner',
                      left_on=target,
                      right_on='zone_id')
    res = res[['zone_id', 'zone_name', 'borough', 'cnt', 'zone_geom']]
    # display(res)
    
    
    return res

In [ ]:
@logging_time
def get_pessenger_with_zone(trg):
    # 1. 두개 table 정보를 모두 읽어 dataframe로 할당한다
    taxi_trips_df = execute_query("SELECT * FROM taxi_trips WHERE pickup_location_id IS NOT NULL AND dropoff_location_id IS NOT NULL;")
    taxi_zone_geom_df = execute_query("SELECT * FROM taxi_zone_geom;")

    #2. taxi_trips_df에서 trg와 passenger_count만을 추출한 후, passenger_count에 대하여 one-hot encoding을 진행한다.
    passenger_one_hot = pd.get_dummies(taxi_trips_df[['passenger_count',trg]], columns = ['passenger_count'])

    #3. passenger_count_0, passenger_count_5, passenger_count_6을 합하여 새로운 column passenger_other을 정의한다.
    #   이후, 불필요한 column을 제거한다.
    passenger_one_hot['passenger_other'] = passenger_one_hot.apply(lambda x : x['passenger_count_0'] + x['passenger_count_5'] + x['passenger_count_6'], axis=1)
    passenger_one_hot = passenger_one_hot.drop(['passenger_count_0', 'passenger_count_5', 'passenger_count_6'], axis=1)

    # 4. passenger_one_hot 테이블을 trg를 기준으로 하여 그룹핑하고, column의 누적합을 구한다.
    passenger_one_hot = passenger_one_hot.groupby(by=trg).sum().reset_index()
    passenger_one_hot = passenger_one_hot.rename(columns={'passenger_count_1' : 'passenger_1', 'passenger_count_2' : 'passenger_2', 'passenger_count_3' : 'passenger_3', 'passenger_count_4' : 'passenger_4'})

    # 5. 해당 구역에서 출발하는 trip에 대하여 passenger의 수별로 차지하는 비유을 구해야하므로, 전체 trip의 수를 구하여 새로운 column `total_trip`을 정의한다.
    passenger_one_hot['total_trip'] = passenger_one_hot.apply(lambda x : x['passenger_1'] + x['passenger_2'] + x['passenger_3'] + x['passenger_4'] + x['passenger_other'], axis=1)

    # 6. passenger에 대한 column을 전체에서 차지하는 비율 값으로 변환한다.
    passenger_one_hot['passenger_1'] = passenger_one_hot.apply(lambda x : x['passenger_1'] / x['total_trip'], axis=1)
    passenger_one_hot['passenger_2'] = passenger_one_hot.apply(lambda x : x['passenger_2'] / x['total_trip'], axis=1)
    passenger_one_hot['passenger_3'] = passenger_one_hot.apply(lambda x : x['passenger_3'] / x['total_trip'], axis=1)
    passenger_one_hot['passenger_4'] = passenger_one_hot.apply(lambda x : x['passenger_4'] / x['total_trip'], axis=1)
    passenger_one_hot['passenger_other'] = passenger_one_hot.apply(lambda x : x['passenger_other'] / x['total_trip'], axis=1)

    # 7. passenger_one_hot과 taxi_zone_geom_df을 join시켜 원하는 출력 형태를 만든다.
    res = passenger_one_hot.merge(taxi_zone_geom_df,
                                how='inner',
                                left_on=trg,
                                right_on='zone_id')
    res = res[['zone_id', 'zone_name', 'borough', 'passenger_1', 'passenger_2', 'passenger_3', 'passenger_4', 'passenger_other', 'zone_geom']]
    # display(res)
    return res

In [ ]:
res = get_pessenger_with_zone('pickup_location_id')
res

WorkingTime[get_pessenger_with_zone]: 37.06279516220093 sec


,zone_id,zone_name,borough,passenger_1,passenger_2,passenger_3,passenger_4,passenger_other,zone_geom
0,1,Newark Airport,EWR,0.682692,0.230769,0.038462,0.038462,0.009615,"POLYGON((-74.1856319999999 40.6916479999999, -..."
1,2,Jamaica Bay,Queens,1.000000,0.000000,0.000000,0.000000,0.000000,MULTIPOLYGON(((-73.8149045978006 40.6104844500...
2,3,Allerton/Pelham Gardens,Bronx,0.600000,0.400000,0.000000,0.000000,0.000000,"POLYGON((-73.848596761 40.8716707849999, -73.8..."
3,4,Alphabet City,Manhattan,0.707731,0.144354,0.045883,0.019485,0.082548,"POLYGON((-73.9717675541021 40.7258333034546, -..."
4,6,Arrochar/Fort Wadsworth,Staten Island,0.833333,0.000000,0.000000,0.000000,0.166667,"POLYGON((-74.0642173399999 40.6023530149999, -..."
...,...,...,...,...,...,...,...,...,...
244,259,Woodlawn/Wakefield,Bronx,1.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON((-73.8514475398451 40.9104496711014, -..."
245,260,Woodside,Queens,0.712074,0.137255,0.024768,0.016512,0.109391,"POLYGON((-73.9022524129999 40.7626438259999, -..."
246,261,World Trade Center,Manhattan,0.685606,0.153317,0.051838,0.029873,0.079367,"POLYGON((-74.013353098 40.7051985769998, -74.0..."
247,262,Yorkville East,Manhattan,0.719217,0.133281,0.036471,0.017232,0.093799,MULTIPOLYGON(((-73.9380672308988 40.7809447032...


In [ ]:
def show_map_range_0_1(df, trg, tooltip_trg):
    """
    입력되는 pandas.DataFrame를 folium의 지도 위도 출력합니다.
    """
    gs = gpd.GeoSeries.from_wkt(df['zone_geom'])
    gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")

    f = folium.Figure(width=600, height=500)
    m = folium.Map(location=[40.75, -73.90], zoom_start=10, tiles='CartoDB positron').add_to(f)
    choropleth = folium.Choropleth(geo_data=gdf, 
                    neme='zone_name',
                    data=gdf,
                    columns=['zone_name', trg],
                    key_on='feature.properties.zone_name',
                    fill_color='YlGn',
                    fill_opacity=0.7,
                    line_opacity=0.2,
                    legend_name=trg,
                    threshold_scale=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1]).add_to(m)

    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}

    tooltip = folium.GeoJson(
        gdf,
        style_function=style_function, 
        control=False,
        tooltip=folium.GeoJsonTooltip(
            fields=tooltip_trg,
            localize=True
        )
    )

    choropleth.add_child(tooltip)
    display(f)

In [ ]:
def show_problem_4_5_map(df):
    for i in range(1, 5):
        print(f'passenger_{i} >>>>>')
        show_map_range_0_1(df[['zone_name', 'borough', f'passenger_{i}','zone_geom']], 
                        f'passenger_{i}', 
                        ['zone_name', 'borough', f'passenger_{i}'])
        print()

In [ ]:
show_problem_4_5_map(res)

Output hidden; open in https://colab.research.google.com to view.

✨ 실습 - 쿼리를 활용하여 문제를 해결해보자!

In [ ]:
query = \
"""
select passenger_one_hot.pickup_location_id,
	   zone.zone_name,
	   zone.borough,
	   passenger_one_hot.passenger_1 / passenger_one_hot.total_trip as passenger_1,
	   passenger_one_hot.passenger_2 / passenger_one_hot.total_trip as passenger_2, 
	   passenger_one_hot.passenger_3 / passenger_one_hot.total_trip as passenger_3,
	   passenger_one_hot.passenger_4 / passenger_one_hot.total_trip as passenger_4,
	   passenger_one_hot.passenger_other / passenger_one_hot.total_trip as passenger_other,
	   zone.zone_geom
from(select pickup_location_id, 
			cast(sum(passenger_1) as real) as passenger_1, 
			cast(sum(passenger_2) as real) as passenger_2, 
			cast(sum(passenger_3) as real) as passenger_3, 
			cast(sum(passenger_4) as real) as passenger_4, 
			cast(sum(passenger_other) as real) as passenger_other,
			cast(count(pickup_datetime) as real) as total_trip
		from (select pickup_location_id, 
					(case when passenger_count=1 then 1 else 0 end) as passenger_1,
					(case when passenger_count=2 then 1 else 0 end) as passenger_2,
					(case when passenger_count=3 then 1 else 0 end) as passenger_3,
					(case when passenger_count=4 then 1 else 0 end) as passenger_4,
					(case when passenger_count NOT IN (1,2,3,4) then 1 else 0 end) as passenger_other,
					pickup_datetime
			  from taxi_trips
			  where pickup_location_id is not null and dropoff_location_id is not null) 
		group by pickup_location_id) as passenger_one_hot
left join taxi_zone_geom as zone
on passenger_one_hot.pickup_location_id=zone.zone_id
order by passenger_one_hot.pickup_location_id
"""

res = solution_with_query(query)
res

WorkingTime[solution_with_query]: 5.1696977615356445 sec


,pickup_location_id,zone_name,borough,passenger_1,passenger_2,passenger_3,passenger_4,passenger_other,zone_geom
0,1,Newark Airport,EWR,0.682692,0.230769,0.038462,0.038462,0.009615,"POLYGON((-74.1856319999999 40.6916479999999, -..."
1,2,Jamaica Bay,Queens,1.000000,0.000000,0.000000,0.000000,0.000000,MULTIPOLYGON(((-73.8149045978006 40.6104844500...
2,3,Allerton/Pelham Gardens,Bronx,0.600000,0.400000,0.000000,0.000000,0.000000,"POLYGON((-73.848596761 40.8716707849999, -73.8..."
3,4,Alphabet City,Manhattan,0.707731,0.144354,0.045883,0.019485,0.082548,"POLYGON((-73.9717675541021 40.7258333034546, -..."
4,6,Arrochar/Fort Wadsworth,Staten Island,0.833333,0.000000,0.000000,0.000000,0.166667,"POLYGON((-74.0642173399999 40.6023530149999, -..."
...,...,...,...,...,...,...,...,...,...
244,259,Woodlawn/Wakefield,Bronx,1.000000,0.000000,0.000000,0.000000,0.000000,"POLYGON((-73.8514475398451 40.9104496711014, -..."
245,260,Woodside,Queens,0.712074,0.137255,0.024768,0.016512,0.109391,"POLYGON((-73.9022524129999 40.7626438259999, -..."
246,261,World Trade Center,Manhattan,0.685606,0.153317,0.051838,0.029873,0.079367,"POLYGON((-74.013353098 40.7051985769998, -74.0..."
247,262,Yorkville East,Manhattan,0.719217,0.133281,0.036471,0.017232,0.093799,MULTIPOLYGON(((-73.9380672308988 40.7809447032...


In [ ]:
show_problem_4_5_map(res)

Output hidden; open in https://colab.research.google.com to view.

### 5) 도착구역에 따른 여행객 수의 비율
- 출력 column : zone_id, zone_name, borough, passenger_1, passenger_2, passenger_3, passenger_4, passenger_other, zone_geom
    - zone_id : 구역의 id
    - zone_name : 구역이 이름
    - borough : 구역의 속하는 지역
    - cnt : 해당 구역에서 출발한 trip의 수
    - passenger_1, passenger_2, passenger_3, passenger_4는 passenger_count에서 오며 각 element는 해당 zone에서 도착한 여행에서 해당 여행객수의 비율(1이하의 float)이다. passenger_other는 1\~4 이외의 여행객 수(0, 5\~9)에 대한 비율이다.

✨ 출력 예시 - 데이터를 모두 읽어와 pandas에서 처리하는 경우

In [ ]:
res = get_pessenger_with_zone('dropoff_location_id')
res

WorkingTime[get_pessenger_with_zone]: 39.0473108291626 sec


,zone_id,zone_name,borough,passenger_1,passenger_2,passenger_3,passenger_4,passenger_other,zone_geom
0,1,Newark Airport,EWR,0.675571,0.176926,0.042199,0.022842,0.082462,"POLYGON((-74.1856319999999 40.6916479999999, -..."
1,2,Jamaica Bay,Queens,1.000000,0.000000,0.000000,0.000000,0.000000,MULTIPOLYGON(((-73.8149045978006 40.6104844500...
2,3,Allerton/Pelham Gardens,Bronx,0.679012,0.209877,0.000000,0.000000,0.111111,"POLYGON((-73.848596761 40.8716707849999, -73.8..."
3,4,Alphabet City,Manhattan,0.716511,0.139573,0.041246,0.018772,0.083897,"POLYGON((-73.9717675541021 40.7258333034546, -..."
4,5,Arden Heights,Staten Island,0.500000,0.375000,0.000000,0.000000,0.125000,"POLYGON((-74.174887504 40.5628264189999, -74.1..."
...,...,...,...,...,...,...,...,...,...
254,259,Woodlawn/Wakefield,Bronx,0.681373,0.127451,0.044118,0.044118,0.102941,"POLYGON((-73.8514475398451 40.9104496711014, -..."
255,260,Woodside,Queens,0.703463,0.153139,0.031926,0.020022,0.091450,"POLYGON((-73.9022524129999 40.7626438259999, -..."
256,261,World Trade Center,Manhattan,0.680375,0.155730,0.046417,0.025249,0.092229,"POLYGON((-74.013353098 40.7051985769998, -74.0..."
257,262,Yorkville East,Manhattan,0.706808,0.142310,0.039297,0.017519,0.094065,MULTIPOLYGON(((-73.9380672308988 40.7809447032...


In [ ]:
show_problem_4_5_map(res)

Output hidden; open in https://colab.research.google.com to view.

✨ 실습 - 쿼리를 활용하여 문제를 해결해보자!

In [ ]:
query = \
"""
select passenger_one_hot.dropoff_location_id as zone_id,
	   zone.zone_name,
	   zone.borough,
	   passenger_one_hot.passenger_1 / passenger_one_hot.total_trip as passenger_1,
	   passenger_one_hot.passenger_2 / passenger_one_hot.total_trip as passenger_2, 
	   passenger_one_hot.passenger_3 / passenger_one_hot.total_trip as passenger_3,
	   passenger_one_hot.passenger_4 / passenger_one_hot.total_trip as passenger_4,
	   passenger_one_hot.passenger_other / passenger_one_hot.total_trip as passenger_other,
	   zone.zone_geom
from(select dropoff_location_id, 
			cast(sum(passenger_1) as real) as passenger_1, 
			cast(sum(passenger_2) as real) as passenger_2, 
			cast(sum(passenger_3) as real) as passenger_3, 
			cast(sum(passenger_4) as real) as passenger_4, 
			cast(sum(passenger_other) as real) as passenger_other,
			cast(count(pickup_datetime) as real) as total_trip
		from (select dropoff_location_id, 
					(case when passenger_count=1 then 1 else 0 end) as passenger_1,
					(case when passenger_count=2 then 1 else 0 end) as passenger_2,
					(case when passenger_count=3 then 1 else 0 end) as passenger_3,
					(case when passenger_count=4 then 1 else 0 end) as passenger_4,
					(case when passenger_count NOT IN (1,2,3,4) then 1 else 0 end) as passenger_other,
					pickup_datetime
			  from taxi_trips
			  where dropoff_location_id is not null and dropoff_location_id is not null) 
		group by dropoff_location_id) as passenger_one_hot
left join taxi_zone_geom as zone
on passenger_one_hot.dropoff_location_id=zone.zone_id
order by passenger_one_hot.dropoff_location_id
"""

res = solution_with_query(query)
res

WorkingTime[solution_with_query]: 5.978572368621826 sec


,zone_id,zone_name,borough,passenger_1,passenger_2,passenger_3,passenger_4,passenger_other,zone_geom
0,1,Newark Airport,EWR,0.675415,0.176766,0.042455,0.022771,0.082594,"POLYGON((-74.1856319999999 40.6916479999999, -..."
1,2,Jamaica Bay,Queens,1.000000,0.000000,0.000000,0.000000,0.000000,MULTIPOLYGON(((-73.8149045978006 40.6104844500...
2,3,Allerton/Pelham Gardens,Bronx,0.679012,0.209877,0.000000,0.000000,0.111111,"POLYGON((-73.848596761 40.8716707849999, -73.8..."
3,4,Alphabet City,Manhattan,0.716528,0.139502,0.041225,0.018890,0.083854,"POLYGON((-73.9717675541021 40.7258333034546, -..."
4,5,Arden Heights,Staten Island,0.500000,0.375000,0.000000,0.000000,0.125000,"POLYGON((-74.174887504 40.5628264189999, -74.1..."
...,...,...,...,...,...,...,...,...,...
254,259,Woodlawn/Wakefield,Bronx,0.682927,0.126829,0.043902,0.043902,0.102439,"POLYGON((-73.8514475398451 40.9104496711014, -..."
255,260,Woodside,Queens,0.703463,0.153139,0.031926,0.020022,0.091450,"POLYGON((-73.9022524129999 40.7626438259999, -..."
256,261,World Trade Center,Manhattan,0.680520,0.155660,0.046396,0.025238,0.092187,"POLYGON((-74.013353098 40.7051985769998, -74.0..."
257,262,Yorkville East,Manhattan,0.706808,0.142310,0.039297,0.017519,0.094065,MULTIPOLYGON(((-73.9380672308988 40.7809447032...


In [ ]:
show_problem_4_5_map(res)

Output hidden; open in https://colab.research.google.com to view.

## 회고
---

### 1) 이번주차의 실습과정에서 알게 된 것은 무엇인가요?
> (내용을 입력하여 주세요)

### 2) 실습과정에서 느낀 pandas로의 데이터처리와 SQL을 이용한 데이터처리의 장단점은 무엇인가요?
> (내용을 입력하여 주세요)

### 3) 느낀점
> (내용을 입력하여 주세요)